In [1]:
# Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import json
import requests

In [24]:
#import main CSV and trim columns to used data
ontimeMarketing = "../Resources/T_ONTIME_MARKETING.csv"

airlineData = pd.read_csv(ontimeMarketing, encoding="utf-8", dtype=object)
cleanAirlineData = airlineData[["ORIGIN_CITY_NAME", "ORIGIN_STATE_ABR", "DEST_CITY_NAME", "DEST_STATE_ABR", "CRS_DEP_TIME", "DEP_TIME", "DEP_DELAY", "TAXI_OUT", "TAXI_IN", "ARR_TIME", "ARR_DELAY", "CANCELLED", "CANCELLATION_CODE", "DIVERTED", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY", "DIV_AIRPORT_LANDINGS", "DIV_ACTUAL_ELAPSED_TIME"]]
cleanAirlineData

,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_TIME,...,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_ACTUAL_ELAPSED_TIME
0,"New York, NY",NY,"Los Angeles, CA",CA,0900,0852,-8.00,18.00,5.00,1124,...,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0,NaN
1,"Los Angeles, CA",CA,"New York, NY",NY,1235,1232,-3.00,17.00,12.00,2107,...,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0,NaN
2,"Boston, MA",MA,"Los Angeles, CA",CA,1800,1754,-6.00,18.00,8.00,2041,...,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0,NaN
3,"Los Angeles, CA",CA,"Boston, MA",MA,0750,0742,-8.00,20.00,15.00,1638,...,0.00,NaN,0.00,0.00,0.00,19.00,0.00,0.00,0,NaN
4,"Phoenix, AZ",AZ,"Portland, OR",OR,1502,1458,-4.00,22.00,5.00,1654,...,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582420,"Phoenix, AZ",AZ,"Los Angeles, CA",CA,1200,1203,3.00,12.00,5.00,1218,...,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0,NaN
582421,"Seattle, WA",WA,"Denver, CO",CO,1800,1842,42.00,15.00,5.00,2218,...,0.00,NaN,0.00,0.00,0.00,6.00,0.00,42.00,0,NaN
582422,"Orlando, FL",FL,"Austin, TX",TX,0705,0703,-2.00,18.00,4.00,0907,...,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0,NaN
582423,"Los Angeles, CA",CA,"Sacramento, CA",CA,1315,1430,75.00,15.00,3.00,1549,...,0.00,NaN,0.00,69.00,0.00,0.00,0.00,0.00,0,NaN


In [ ]:
"""
GeoAPIfy integration here for extra credit
"""

In [25]:
# filter down to Denver flights only, and fill null values
denverAirlines = cleanAirlineData.loc[(cleanAirlineData["ORIGIN_CITY_NAME"] == "Denver, CO") | (cleanAirlineData["DEST_CITY_NAME"] == "Denver, CO")].fillna(0.0).reset_index(drop=True)
denverAirlines

,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_TIME,...,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DIV_ACTUAL_ELAPSED_TIME
0,"Denver, CO",CO,"Phoenix, AZ",AZ,1206,1204,-2.00,13.00,4.00,1355,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
1,"Phoenix, AZ",AZ,"Denver, CO",CO,0930,0926,-4.00,16.00,6.00,1110,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
2,"Dallas/Fort Worth, TX",TX,"Denver, CO",CO,2245,2244,-1.00,17.00,31.00,0002,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
3,"Dallas/Fort Worth, TX",TX,"Denver, CO",CO,1027,1041,14.00,27.00,10.00,1155,...,0.00,0.0,0.00,14.00,0.00,3.00,0.00,0.00,0,0.0
4,"Denver, CO",CO,"Dallas/Fort Worth, TX",TX,1240,1247,7.00,12.00,6.00,1531,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48992,"Salt Lake City, UT",UT,"Denver, CO",CO,1515,1515,0.00,13.00,10.00,1635,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
48993,"Tampa, FL",FL,"Denver, CO",CO,2045,2038,-7.00,7.00,7.00,2222,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
48994,"Denver, CO",CO,"Santa Ana, CA",CA,0835,0828,-7.00,12.00,6.00,0940,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0
48995,"Houston, TX",TX,"Denver, CO",CO,0625,0620,-5.00,12.00,7.00,0736,...,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0


In [26]:
# counts of delay reasons
delayedFlights = denverAirlines.loc[(denverAirlines["DEP_DELAY"].astype(float) > 0.0) | (denverAirlines["ARR_DELAY"].astype(float) > 0.0)]
delayedFlights = delayedFlights[["ORIGIN_CITY_NAME", "ORIGIN_STATE_ABR", "DEST_CITY_NAME", "DEST_STATE_ABR", "DEP_DELAY", "ARR_DELAY", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]]
delayedFlights

,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_CITY_NAME,DEST_STATE_ABR,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2,"Dallas/Fort Worth, TX",TX,"Denver, CO",CO,-1.00,17.00,31.00,12.00,0.0,0.0,0.0,0.0,0.0
3,"Dallas/Fort Worth, TX",TX,"Denver, CO",CO,14.00,27.00,10.00,17.00,14.00,0.00,3.00,0.00,0.00
4,"Denver, CO",CO,"Dallas/Fort Worth, TX",TX,7.00,12.00,6.00,-7.00,0.0,0.0,0.0,0.0,0.0
5,"Denver, CO",CO,"Dallas/Fort Worth, TX",TX,-2.00,34.00,12.00,1.00,0.0,0.0,0.0,0.0,0.0
6,"Charlotte, NC",NC,"Denver, CO",CO,43.00,10.00,6.00,10.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48970,"San Francisco, CA",CA,"Denver, CO",CO,26.00,30.00,6.00,31.00,0.00,0.00,5.00,0.00,26.00
48977,"St. Louis, MO",MO,"Denver, CO",CO,3.00,10.00,7.00,-21.00,0.0,0.0,0.0,0.0,0.0
48982,"Detroit, MI",MI,"Denver, CO",CO,7.00,17.00,5.00,-6.00,0.0,0.0,0.0,0.0,0.0
48986,"Denver, CO",CO,"Orlando, FL",FL,11.00,16.00,12.00,12.00,0.0,0.0,0.0,0.0,0.0
